# Selecting Logan Search results by Sequencing Technology using Python in Jupyter

This notebook demonstrates how you could filter Logan Search Results by technolgy.

This assumes you have worked through Working with with Logan Search results in Jupyter](Working_with_Logan_Search_results_with_Python_in_Jupyter.ipynb). If you haven't yet, please do that first.   
Indeed the first code cell below is directly from the first cell [there](Working_with_Logan_Search_results_with_Python_in_Jupyter.ipynb).


----------

#### Reading in the results for several queries

Run the code in the next cell assigning the settings to match the set-up preparation performed and to bring in the results data to the active namespace:


In [ ]:
results_dir = "/home/jovyan/logan_search_results/"
consistent_part_of_file_name_suffix = "_31mer_"

import os
import fnmatch
import pandas as pd

def extract_tags_from_filename(filename):
    '''
    extract tag labels from file names

    return the tag label
    '''
    first_part = filename.split(consistent_part_of_file_name_suffix,1)[0]
    all_but_first_part_of_remainder_as_list = first_part.split("_")[1:]
    return "_".join(all_but_first_part_of_remainder_as_list)
    
matched_filenames = []
file_tags_per_file = []
dfs = []
for file in os.listdir(results_dir):
    if fnmatch.fnmatch(file, '*.tsv'):
        matched_filenames.append(file)
        file_tags_per_file.append(extract_tags_from_filename(file))
        dfs.append(pd.read_csv(results_dir + file, sep="\t"))

# provide feedback about what was read in
max_filename_length = max(len(filename) for filename in matched_filenames)
for filename, df in zip(matched_filenames, dfs): 
    print(f'from {filename}:{" " * (max_filename_length - len(filename))} {len(df):>6} rows read in')

#print(file_tags_per_file) # FOR DEBUGGING, uncomment
#print(matched_filenames) # FOR DEBUGGING, uncomment
gene_name = file_tags_per_file[0].split("_",1)[0]
# make tuples of tag labels and corresponding dataframes
tags_and_dataframes = list(zip(file_tags_per_file, dfs)) # tags are first in each two-item tuple, the dataframe is second
# make dictionary of tuples of tag labels and corresponding dataframes, integers will be the keys with the tuples as values
tags_and_dataframes_dict = dict(zip(range(len(file_tags_per_file)), zip(file_tags_per_file, dfs))) # integers ('index') are keys with tags are first in each two-item tuple, the dataframe is second
print(f"\nTOTAL QUERY RESULTS READ IN: {len(file_tags_per_file)}")

--------

## Selecting based on Sequencing Technology

#### Step #1 add a type column based on Instrument used

In step #1, we'll add and additional column to the data based on the `Instruments` column.  
To do that first we have to bring in a table that has the technology instruments 



In [ ]:
Now we are ready to do tthe assignment step.

If you later edit this notebook to repeat it with your own results, keep in mind tha assignment step may fail. Whike an effort was made to make it fairly extensive, the table may not be complete at this time. If you see an error here it may be that there is no match for an instrument listed in your results among the list supplied here. To fix it though, you should be able to edit the comma-separated values file [there](supporting_tables/sequencing_instrument_details_for_all_SRA_options_seen_thus_far.csv),save it, and run the code again.

In [ ]:
#### Step #2 Sort on the technology type